# AIカメラで人を認識させよう

AIカメラをモノ認識(Object Recognition)モードにすることで、人を認識することができます。すでに人の形状が覚えられていますが、ID：１として人を認識するには、カメラを人に向けてを学習させる必要があります。

カメラをObject Recognitionモードに切り替えて、人を写した状態で右上のボタンを押して学習させてください。人を学習できると人を写すとID:1と表示されます。AIカメラからは<br>
 [('block', 1, 282, 125, 70, 184)]<br>
 という形式で返却されます。２番目の数字、1 が　ID:1を見つけたことを示しています。後半の数字は、人を見つけた位置 (x, y) とサイズ (w, h) です。

AIカメラとLEGOを組みあわせて、人を正しく認識できるかテストしましょう。以下のプログラムはAIカメラをObject Recognision(モノ　認識モード)に設定して、人を探します。人を見つけられるとＡＩカメラはＩＤ：１を返却します。LEGOはスマイルを表示して音を鳴らします。どの程度離れていたら人と認識できるのか、どんな姿勢だと認識できるのか試しましょう。もし、人を見つけられない場合は、再度学習させてください。


In [ ]:
import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

from spike import PrimeHub

prime_hub = PrimeHub()        # HUBの初期化

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

husky.send_CMD_REQ_ALGO(Algo.OBJECT_RECOGNITION)
time.sleep(1)

while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.off()
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=80, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')

上記プログラムにより、人と認識すると、音を鳴らして、LEDに笑顔を表示します。うまく人をみつけられたでしょうか。

# 改良：人に近づくプログラム

犬を飼っていると、人を見つけられたら近寄ってきますよね？　LEGOでも同じ動きをさせてみましょう。人を見つけられたら、サーボを使って前進させましょう。

以前プログラムしましたが、サーボを使ってLEGOを前進させるプログラムは以下です。これは50cm進んで止まるプログラムです。もっと進ませる場合は、距離を変えることで、進む距離を変えることができます。

In [ ]:
from spike import MotorPair

motor_pair = MotorPair('A','B')

motor_pair.set_motor_rotation(17.6, 'cm')            #  タイヤのサイズを指定します
motor_pair.move(50, unit='cm', steering=0,speed=40)  # 50cmまっすぐ進むようmove関数を実行

先ほど作った人を見つけるプログラムと、サーボを使って前進するプログラムを組み合わせて、人を見つけたら 50cm 進むように改造しましょう。

In [ ]:
import sys
import time
from hub import port
from spike import MotorPair   #  モータを使うので追加


sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

from spike import PrimeHub

prime_hub = PrimeHub()        # HUBの初期化
motor_pair = MotorPair('A','B')   # モータペアの初期化　（追加）
motor_pair.set_motor_rotation(17.6, 'cm')    #  タイヤのサイズを指定します

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

husky.send_CMD_REQ_ALGO(Algo.OBJECT_RECOGNITION)
time.sleep(1)

while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.off()
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=80, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
            motor_pair.move(50, unit='cm', steering=0,speed=40)  # 50cmまっすぐ進むようmove関数を実行

うまく動きましたか？　人を見つけるとスマイルマークを表示して前進します。人を見つけると、　５０ｃｍ進む動きをします。